In [108]:
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

In [112]:
df_budget = pd.read_csv("df_budget.csv")
df_data=pd.read_csv("df_Data.csv")
display(df_budget.head())
display(df_data.head())


,TIPO,REF,Etapa,ACCT,BUDGET
0,Above the Line,Cuenta General,NaN,1100,"10,065,000.00"
1,Above the Line,Sub cuenta,NaN,1101,"8,340,000.00"
2,Above the Line,Detalle,NaN,1101-002,0.00
3,Above the Line,Detalle,NaN,1101-004,0.00
4,Above the Line,Detalle,DEVELOPMENT,1101-005,"1,400,000.00"


,SUBTOTAL,PARTIDA PRESUPUESTAL
0,"7,000,000.00",Income (Funding)
1,"1,120,000.00",Income (Funding) IVA
2,"700,000.00",Income (Funding)
3,"14,182.00",7007-009
4,"110,000.00",1107-009


In [113]:
df_data = df_data.rename(columns={"PARTIDA PRESUPUESTAL": "ACCT"})
df_data


,SUBTOTAL,ACCT
0,"7,000,000.00",Income (Funding)
1,"1,120,000.00",Income (Funding) IVA
2,"700,000.00",Income (Funding)
3,"14,182.00",7007-009
4,"110,000.00",1107-009
...,...,...
7881,NaN,NaN
7882,NaN,NaN
7883,NaN,NaN
7884,NaN,NaN


In [116]:
df_data = df_data.groupby("ACCT", as_index=False)[["SUBTOTAL"]].sum()
df_data

,ACCT,SUBTOTAL
0,1101-005,"1,400,000.00"
1,1101-006,"100,000.03"
2,1101-007,"100,000.00"
3,1101-011,"1,400,000.00"
4,1101-012,"75,000.00"
...,...,...
1053,Income (Funding),"134,862,103.57"
1054,Income (Funding) IVA,"19,352,891.03"
1055,PC El Niñero S01,"241,795.83"
1056,Pago Carolina Rivera,"1,000,000.00"


In [117]:
df_data[df_data["ACCT"] == "1104-003"]

,ACCT,SUBTOTAL
16,1104-003,"150,000.00"


In [118]:
df_CostReport1=pd.merge(df_budget,df_data, on="ACCT", how="outer")
df_CostReport2= df_CostReport1.rename(columns = {"SUBTOTAL": "COST TO DATE"})
df_CostReport2

,TIPO,REF,Etapa,ACCT,BUDGET,COST TO DATE
0,Above the Line,Cuenta General,NaN,1100,"10,065,000.00",NaN
1,Above the Line,Sub cuenta,NaN,1101,"8,340,000.00",NaN
2,Above the Line,Detalle,NaN,1101-002,0.00,NaN
3,Above the Line,Detalle,NaN,1101-004,0.00,NaN
4,Above the Line,Detalle,DEVELOPMENT,1101-005,"1,400,000.00","1,400,000.00"
...,...,...,...,...,...,...
2482,NaN,NaN,NaN,Income (Funding),NaN,"134,862,103.57"
2483,NaN,NaN,NaN,Income (Funding) IVA,NaN,"19,352,891.03"
2484,NaN,NaN,NaN,PC El Niñero S01,NaN,"241,795.83"
2485,NaN,NaN,NaN,Pago Carolina Rivera,NaN,"1,000,000.00"


In [119]:
df_CostReport2[df_CostReport2["ACCT"] == "1104-003"]

,TIPO,REF,Etapa,ACCT,BUDGET,COST TO DATE
31,Above the Line,Detalle,DEVELOPMENT,1104-003,"150,000.00","150,000.00"


In [120]:

df_CostReport3 = df_CostReport2.groupby("ACCT", as_index=False).agg({
    "TIPO": "first",
    "REF": "first",
    "Etapa": "first",
    "BUDGET": "first",
    "COST TO DATE": "sum"
})
df_CostReport3


,ACCT,TIPO,REF,Etapa,BUDGET,COST TO DATE
0,1100,Above the Line,Cuenta General,None,"10,065,000.00",0.00
1,1101,Above the Line,Sub cuenta,None,"8,340,000.00",0.00
2,1101-002,Above the Line,Detalle,None,0.00,0.00
3,1101-004,Above the Line,Detalle,None,0.00,0.00
4,1101-005,Above the Line,Detalle,DEVELOPMENT,"1,400,000.00","1,400,000.00"
...,...,...,...,...,...,...
2482,Income (Funding),None,None,None,NaN,"134,862,103.57"
2483,Income (Funding) IVA,None,None,None,NaN,"19,352,891.03"
2484,PC El Niñero S01,None,None,None,NaN,"241,795.83"
2485,Pago Carolina Rivera,None,None,None,NaN,"1,000,000.00"


In [121]:
df_CostReport3[df_CostReport3["ACCT"] == "1104-003"]


,ACCT,TIPO,REF,Etapa,BUDGET,COST TO DATE
31,1104-003,Above the Line,Detalle,DEVELOPMENT,"150,000.00","150,000.00"


In [132]:
df_CostReport3["VARIANCE"] = df_CostReport3["BUDGET"] - df_CostReport3["COST TO DATE"]
df_CostReport4 = df_CostReport3.groupby("ACCT", as_index=False).agg({
    "TIPO": "first",
    "REF": "first",
    "Etapa": "first",
    "BUDGET": "first",
    "COST TO DATE": "sum"
})

# Agregar la columna de diferencia
df_CostReport4["VARIANCE"] = df_CostReport4["BUDGET"] - df_CostReport4["COST TO DATE"]

# Mostrar el resultado
df_CostReport4.head(15)

,ACCT,TIPO,REF,Etapa,BUDGET,COST TO DATE,VARIANCE
0,1100,Above the Line,Cuenta General,None,"10,065,000.00",0.00,"10,065,000.00"
1,1101,Above the Line,Sub cuenta,None,"8,340,000.00",0.00,"8,340,000.00"
2,1101-002,Above the Line,Detalle,None,0.00,0.00,0.00
3,1101-004,Above the Line,Detalle,None,0.00,0.00,0.00
4,1101-005,Above the Line,Detalle,DEVELOPMENT,"1,400,000.00","1,400,000.00",0.00
5,1101-006,Above the Line,Detalle,DEVELOPMENT,"100,000.00","100,000.03",-0.03
6,1101-007,Above the Line,Detalle,DEVELOPMENT,"100,000.00","100,000.00",0.00
7,1101-010,Above the Line,Detalle,None,0.00,0.00,0.00
8,1101-011,Above the Line,Detalle,DEVELOPMENT,"1,400,000.00","1,400,000.00",0.00
9,1101-012,Above the Line,Detalle,DEVELOPMENT,"75,000.00","75,000.00",0.00


In [133]:
df_CostReport4[df_CostReport3["ACCT"] == "1104-003"]

,ACCT,TIPO,REF,Etapa,BUDGET,COST TO DATE,VARIANCE
31,1104-003,Above the Line,Detalle,DEVELOPMENT,"150,000.00","150,000.00",0.00


In [135]:
import pandas as pd

# Asegúrate de que la columna ACCT es string
df_CostReport4["ACCT"] = df_CostReport4["ACCT"].astype(str)

# Filtro por Detalle
detalle_df = df_CostReport4[df_CostReport4["REF"] == "Detalle"].copy()

# --- Calcular SUBCUENTAS (4 primeros caracteres del código Detalle) ---
sub_df = detalle_df.copy()
sub_df["ACCT"] = sub_df["ACCT"].str[:4]  # Subcuenta: primeros 4 caracteres
sub_sums = sub_df.groupby("ACCT", as_index=False)[["BUDGET", "COST TO DATE"]].sum()
sub_sums["REF"] = "Sub cuenta"
sub_sums["TIPO"] = "Above the Line"

# --- Calcular CUENTA GENERAL (2 primeros caracteres del código Subcuenta) ---
# NOTA: usamos las subcuentas agregadas para hacer el total de cuenta general
gen_df = sub_sums.copy()
gen_df["ACCT"] = gen_df["ACCT"].str[:2] + "00"  # Cuenta General: ejemplo 11 -> 1100
gen_sums = gen_df.groupby("ACCT", as_index=False)[["BUDGET", "COST TO DATE"]].sum()
gen_sums["REF"] = "Cuenta General"
gen_sums["TIPO"] = "Above the Line"

# --- Unir todo ---
final_df = pd.concat([gen_sums, sub_sums, detalle_df], ignore_index=True)

# --- Calcular VARIANCE ---
final_df["VARIANCE"] = final_df["BUDGET"] - final_df["COST TO DATE"]

# --- Mostrar sin notación científica ---
pd.set_option('display.float_format', '{:,.2f}'.format)

# Ver resultado
final_df.sort_values(by="ACCT")


,ACCT,BUDGET,COST TO DATE,REF,TIPO,Etapa,VARIANCE
0,1100,"10,065,000.00","10,065,000.08",Cuenta General,Above the Line,NaN,-0.08
35,1101,"8,340,000.00","8,340,000.02",Sub cuenta,Above the Line,NaN,-0.02
244,1101-002,0.00,0.00,Detalle,Above the Line,None,0.00
245,1101-004,0.00,0.00,Detalle,Above the Line,None,0.00
246,1101-005,"1,400,000.00","1,400,000.00",Detalle,Above the Line,DEVELOPMENT,0.00
...,...,...,...,...,...,...,...
2473,7230-001,0.00,"7,050.76",Detalle,Others,None,"-7,050.76"
2474,7230-002,0.00,"11,436.91",Detalle,Others,None,"-11,436.91"
2475,7230-003,0.00,"12,178.92",Detalle,Others,None,"-12,178.92"
2476,7230-004,0.00,"3,025.87",Detalle,Others,None,"-3,025.87"


In [136]:
final_df.to_csv("df_CostRepor.csv", index=False)